### API docs for the WikiSRATMicroService
##### This service will not work properly for the pollution assessment

#### SRAT code is all in SQL, run from this notebook. This code is within the GitHub repository. The result of this will be a table of NHDplus catchments. 


In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import os
import psycopg2
from StringParser import StringParser
from DatabaseAdapter import DatabaseAdapter
from DatabaseFormatter import DatabaseFormatter

In [13]:
# RAPLACE THIS WITH ACCESS INFORMATION
_database = ''
_host = ''
_port = ''
_user = ''
_password = ''

In [4]:
# FOR HUC12s
# The routine flag can either be 'base' or 'restoration'
_flag = 'base'

def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.args[0] if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
    }

def lambda_handler(event, context):
    try:
        data = StringParser.parse(event['body'])
        db = DatabaseAdapter(_database, _user, _host, _port, _password, _flag)
        input_array = DatabaseAdapter.python_to_array(data)
        return respond(None, db.run_model(input_array))
    except AttributeError as e:
        return respond(e)

_PG_Connection = psycopg2.connect(
                host=_host,
                database=_database,
                user=_user,
                password=_password,
                port= _port)

_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw order by huc12;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata))

_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

_r = dict(lambda_handler({"body": _body},None))
print('done')


484
base
done


In [10]:
print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

{'4481881': {'comid': 4481881, 'tploadrate_total': 8.67678906666698, 'tploadate_conc': 0.000804725157152192, 'tnloadrate_total': 205.478173246534, 'tnloadrate_conc': 0.019056986863077, 'tssloadrate_total': 12430.9823731758, 'tssloadrate_conc': 1.1529062383503}, '4481681': {'comid': 4481681, 'tploadrate_total': 45.9269178229512, 'tploadate_conc': 0.0139356826495012, 'tnloadrate_total': 1098.12798966065, 'tnloadrate_conc': 0.333206840298741, 'tssloadrate_total': 128512.512354698, 'tssloadrate_conc': 38.9947698116636}, '4481279': {'comid': 4481279, 'tploadrate_total': 9.3369485692889, 'tploadate_conc': 0.00622679932392672, 'tnloadrate_total': 260.499630512306, 'tnloadrate_conc': 0.173726877803797, 'tssloadrate_total': 4135.72100063273, 'tssloadrate_conc': 2.75810716312542}, '4481935': {'comid': 4481935, 'tploadrate_total': 47.8207522003034, 'tploadate_conc': 0.107891339771179, 'tnloadrate_total': 1122.82652578667, 'tnloadrate_conc': 2.53327797292518, 'tssloadrate_total': 29233.9090668937,

In [6]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (REQUIRES HIGHER PERMISSION)
# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

tablename = 'z_testoutput_base'

len(dict(json.loads(_r['body']))['huc12s'])
_nhdloads = dict(json.loads(_r['body']))['huc12s']
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershed.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6])) 
print('done')

UniqueViolation: duplicate key value violates unique constraint "pk_z_testoutput_base"
DETAIL:  Key (comid)=(2612952) already exists.


In [7]:
_flag = 'restoration'

_r = dict(lambda_handler({"body": _body},None))
print('done')

restoration
done


In [11]:
# print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

In [12]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (REQUIRES HIGHER PERMISSION)
# LOADING RESULTS INTO THE DB CAN TAKE ~10 MINUTES

tablename = 'z_testoutput_restoration'

len(dict(json.loads(_r['body']))['huc12s'])
_nhdloads = dict(json.loads(_r['body']))['huc12s']
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershed.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6])) 
print('done')

done


# END OF USEFUL CODE

In [ ]:
_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw_huc08 order by huc08;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata[0][1]))


In [ ]:
_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw_huc08 where area_sqkm is not null;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
_dbdata

_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

_r = dict(lambda_handler({"body": _body},None))
print('done')